In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp

# (a) Optimum of LS problem
Let $g(x)=||\tilde{y}-y||_2^2$, then we have $\nabla g(x)=-2H^T\tilde{y}-Hx=0$ and $\nabla^2 g(x)=2H^TH>0$, which leads to the solution of question: 

$$ \hat{x}=(H^TH)^{-1}H^T\tilde{y} $$

which is the optimum of the solution.

# (b) return the matrix of H

In [2]:
def matrix_return(t,theta):
    H = []
    for i in range(len(theta)):
        H.append(np.cos(theta[i]*t))
    return np.transpose(H)

# (c)
## Aronson’s sequence

In [3]:
y_tilde_data = pd.read_csv('mathias_distress_call_1.csv')
y_tilde = y_tilde_data.values
y_tilde_data_2 = pd.read_csv('mathias_distress_call_2.csv')
y_tilde_2 = y_tilde_data_2.values

dt = 1.0/8192
t = np.arange(0,len(y_tilde)*dt,dt)
theta = [104, 111, 116, 122, 126, 131, 136, 142, 147, 158, 164, 169, 
         174, 181, 183, 193, 199, 205, 208, 214, 220, 226, 231, 237, 
         243, 249, 254]

H = matrix_return(t,theta)
x_hat_i = np.linalg.solve(np.dot(np.transpose(H),H),np.dot(np.transpose(H),y_tilde))
print('residual error:', np.linalg.norm(y_tilde-np.dot(H,x_hat_i)))

residual error: 2046.298989299373


## Mathias own musical scale

In [4]:
theta = np.arange(311.127, 311.127+311.127,311.127/27)
H = matrix_return(t,theta)
x_hat_ii = np.linalg.solve(np.dot(np.transpose(H),H),np.dot(np.transpose(H),y_tilde))
print('residual error:', np.linalg.norm(y_tilde-np.dot(H,x_hat_ii)))

residual error: 1963.530567863447


## "Fibonacci-on-steroids" sequence

In [5]:
theta = np.zeros(27)
theta[0] = 150
theta[1] = 175
for i in range(25):
    theta[i+2] = np.ceil(0.5*theta[i+1])+np.ceil(0.8*theta[i])
H = matrix_return(t,theta)
x_hat_iii = np.linalg.solve(np.dot(np.transpose(H),H),np.dot(np.transpose(H),y_tilde))
print('residual error:', np.linalg.norm(y_tilde-np.dot(H,x_hat_iii)))

residual error: 983.5818503735173


Based on the residual error, we know "Fibonacci-on-steroids" sequences works the best. We find from the solution that some values are very close to zero. We round the solution and find that this solution is sparse and only $a_5$, $a_7$, $a_8$, $a_{18}$, $a_{19}$, $a_{20}$, $a_{21}$, $a_{27}$ are nonzero, these values are printed below. Therefore, the message only contains the frequency of $\theta_5$, $\theta_7$, $\theta_8$, $\theta_{18}$, $\theta_{19}$, $\theta_{20}$, $\theta_{21}$, $\theta_{27}$

In [6]:
round_ = np.rint(x_hat_iii)
print('(integer) amplitudes of existing frequencies:',round_[round_.nonzero()])

(integer) amplitudes of existing frequencies: [2. 6. 3. 4. 1. 7. 6. 1.]


# (d)
Based on encoding rules, and to make the message meaningful, I believe the message is "Sehr Gut"

# (e) 

## hello

In [ ]:
lambd = 0.0
x_cp = cp.Variable(27)
obj = cp.Minimize(cp.sum_squares(H @ x_cp - y_tilde[:,0]) + lambd * cp.norm(x_cp,1))
prob = cp.Problem(obj)
prob.solve()
print("status:", prob.status)
print("optimal value", prob.value)

(40960,)